# Assignment 2 - Recurrent Neural Networks



## Programming (Full points: 100)

In this assignment, our goal is to use PyTorch to implement Recurrent Neural Networks (RNN) for sentiment analysis task. Sentiment analysis is to classify sentences (input) into certain sentiments (output labels), which includes positive, negative and neutral.

We will use a benckmark dataset, SST, for this assignment.
* we download the SST dataset from torchtext package, and do some preprocessing to build vocabulary and split the dataset into training/validation/test sets. You don't need to modify the code in this step.


In [8]:
import copy
import torch
from torch import nn
from torch import optim

import torchtext
from torchtext import data
from torchtext import datasets

# Set device for PyTorch operations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.LabelField()

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL)

# build dictionary
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

# hyperparameters
vocab_size = len(TEXT.vocab)
label_size = len(LABEL.vocab)
padding_idx = TEXT.vocab.stoi['<pad>']
embedding_dim = 128
hidden_dim = 128

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size=32)

* define the training and evaluation function in the cell below.
### (25 points)


In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    
    for batch in iterator:
        # Zero gradients
        optimizer.zero_grad()
        
        # Get text and labels, move to device
        text = batch.text.to(device)
        labels = batch.label.to(device)
        
        # Forward pass
        predictions = model(text)
        
        # Calculate loss
        loss = criterion(predictions, labels)
        
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        # Accumulate loss
        epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        for batch in iterator:
            # Get text and labels, move to device
            text = batch.text.to(device)
            labels = batch.label.to(device)
            
            # Forward pass
            predictions = model(text)
            
            # Calculate loss
            loss = criterion(predictions, labels)
            epoch_loss += loss.item()
            
            # Calculate accuracy
            predicted_labels = predictions.argmax(dim=1)
            correct_predictions += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)
    
    accuracy = correct_predictions / total_samples
    avg_loss = epoch_loss / len(iterator)
    
    return avg_loss, accuracy

* build a RNN model for sentiment analysis in the cell below.
We have provided several hyperparameters we needed for building the model, including vocabulary size (vocab_size), the word embedding dimension (embedding_dim), the hidden layer dimension (hidden_dim), the number of layers (num_layers) and the number of sentence labels (label_size). Please fill in the missing codes, and implement a RNN model.
### (40 points)

In [10]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, padding_idx):
        super(RNNClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.label_size = label_size
        self.num_layers = 1

        # add the layers required for sentiment analysis.
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=padding_idx)
        self.rnn = nn.RNN(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )
        self.fc = nn.Linear(self.hidden_dim, self.label_size)

    def zero_state(self, batch_size):
        # return initial hidden state on the correct device
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)

    def forward(self, text):
        # text: [batch_size, seq_len]
        embedding = self.embedding(text)
        hidden = self.zero_state(text.size(0))
        outputs, hidden = self.rnn(embedding, hidden)
        final_hidden = hidden[-1]
        logits = self.fc(final_hidden)
        return logits

* train the model and compute the accuracy in the cell below.
### (20 points)

In [11]:
# train baseline RNN model and report accuracies
model = RNNClassifier(vocab_size, embedding_dim, hidden_dim, label_size, padding_idx).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 5
best_val_acc = 0.0
best_state = None

for epoch in range(num_epochs):
    train_loss = train(model, train_iter, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_iter, criterion)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_state = copy.deepcopy(model.state_dict())
    print(f"Epoch {epoch+1}/{num_epochs} | train loss: {train_loss:.4f} | val loss: {val_loss:.4f} | val acc: {val_acc:.4f}")

# load best validation model before testing
if best_state is not None:
    model.load_state_dict(best_state)

test_loss, test_acc = evaluate(model, test_iter, criterion)
print(f"Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")


Epoch 1/5 | train loss: 1.0545 | val loss: 1.0457 | val acc: 0.5395
Epoch 2/5 | train loss: 1.0475 | val loss: 1.1039 | val acc: 0.4069
Epoch 3/5 | train loss: 1.0461 | val loss: 1.1084 | val acc: 0.3797
Epoch 4/5 | train loss: 1.0454 | val loss: 1.0516 | val acc: 0.5268
Epoch 5/5 | train loss: 1.0446 | val loss: 1.2143 | val acc: 0.3097
Test loss: 1.0564 | Test acc: 0.5077


* try to train a model with better accuracy in the cell below. For example, you can use different optimizers such as SGD and Adam. You can also compare different hyperparameters and model size.
### (15 points), to obtain FULL point in this problem, the accuracy needs to be higher than 70%